# SQL 작성

In [1]:
import sys
import os
import pandas as pd
import cv2
from glob import glob
sys.path.append('../../src')
import DISData as DD

## SQL 연결

In [2]:
doUT = DD.SQL()

host:  192.168.0.201
user:  diadmin
password:  ········


SQL connection done


## query 작성

In [3]:
query1 = (f'''CREATE TABLE IF NOT EXISTS celebA_attrib_info (
        id INT(11) NOT NULL AUTO_INCREMENT primary key,
        img_name VARCHAR(30) NOT NULL,
        img_dir VARCHAR(100) NOT NULL,
        img_format VARCHAR(10) NOT NULL,
        img_width INT(11) NOT NULL,
        img_height INT(11) NOT NULL,
        color_space VARCHAR(10) NOT NULL,
        label_dir VARCHAR(100) NOT NULL,
        label_format VARCHAR(10) NOT NULL,
        label VARCHAR(30) NOT NULL,
        label_id INT(10) NOT NULL,
        value VARCHAR(30) NOT NULL)''')

In [8]:
query2 = (f'''INSERT INTO celebA_attrib_info
        (img_name, img_dir, img_format, img_width, img_height,
        color_space, label_dir, label_format, label, label_id,
        value) values(%s, %s, %s, %s, %s,  %s, %s, %s, %s, %s,  %s)''')

## qurey 실행

In [5]:
try:
    doUT.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [6]:
categories = {
    'face_type': ['Oval_Face','Pointy_Nose','Pale_Skin','Rosy_Cheeks','Receding_Hairline','Double_Chin'],
    'hair_style': ['Bangs','Bald','Black_Hair','Blond_Hair','Brown_Hair','Gray_Hair','Straight_Hair','Wavy_Hair'],
    'Facial_Features': ['Arched_Eyebrows','Big_Nose','Bushy_Eyebrows','Narrow_Eyes','Chubby','High_Cheekbones','Smiling','Mouth_Slightly_Open'],
    'accessory': ['Eyeglasses','Wearing_Earrings','Wearing_Hat','Wearing_Lipstick','Wearing_Necklace','Wearing_Necktie'],
    'facial_hair': ['Shadow_of_5_o_clock','No_Beard','Goatee','Mustache','Sideburns'],
    'style': ['Heavy_Makeup','Attractive'],
    'etc': ['Blurry']
}

In [9]:
label_path = []
label_name= []
value_list = []

# label 경로
label_id_path = 'Z:\\celebA\\data\\RGB\\labeldata\\img_celeba\\identity_CelebA.txt'
label_attr_path = 'Z:\\celebA\\data\\RGB\\labeldata\\img_celeba\\list_attr_celeba.txt'
image_path = 'Z:\\celebA\\data\\RGB\\rawdata\\img_celeba'

error_list = []
count = 0

with open(label_id_path, 'r') as f:
    id_texts = f.readlines()

attr_texts = pd.read_csv(label_attr_path, delimiter=' ')

# print(attr_texts)

for i in range(len(id_texts)):
    id_text = id_texts[i].split('\n')[0]
    img_name, label_id = id_text.split(' ')
    color_space = 'RGB'
    img_dir = os.path.join(image_path, label_id, img_name)
    img_file = os.path.basename(img_dir)
    label_dir = label_attr_path
    label_name, label_format = os.path.splitext(os.path.basename(label_dir))
    img_name, img_format = os.path.splitext(os.path.basename(img_dir))
    label_format = label_format.replace('.', '')
    img = cv2.imread(img_dir)
    img_w = img.shape[0]
    img_h = img.shape[1]

    # 각 카테고리에 대해 label 값을 확인하고 1인 경우에만 데이터 삽입
    for category, labels in categories.items():
        for label in labels:
            if attr_texts[label][i] == 1:
                
                value = label  # label이 1인 경우 value는 1로 설정
                values = (img_name, img_dir, img_format, img_w, img_h, color_space, label_dir, label_format,
                          category, label_id, value)
                value_list.append(values)

                if len(value_list) >= 500:
                    print(value_list[0])
                    doUT.insert_dataset_values(query2, value_list)
                    value_list = []
                    print(i, len(id_texts))

doUT.insert_dataset_values(query2, value_list)
# print(value_list2)
value_list = []
print('done')

('000001', 'Z:\\celebA\\data\\RGB\\rawdata\\img_celeba\\2880\\000001.jpg', '.jpg', 687, 409, 'RGB', 'Z:\\celebA\\data\\RGB\\labeldata\\img_celeba\\list_attr_celeba.txt', 'txt', 'face_type', '2880', 'Pointy_Nose')
insert 500 done
66 202599
('000067', 'Z:\\celebA\\data\\RGB\\rawdata\\img_celeba\\1175\\000067.jpg', '.jpg', 420, 640, 'RGB', 'Z:\\celebA\\data\\RGB\\labeldata\\img_celeba\\list_attr_celeba.txt', 'txt', 'Facial_Features', '1175', 'Big_Nose')
insert 500 done
131 202599
('000132', 'Z:\\celebA\\data\\RGB\\rawdata\\img_celeba\\5173\\000132.jpg', '.jpg', 1805, 1280, 'RGB', 'Z:\\celebA\\data\\RGB\\labeldata\\img_celeba\\list_attr_celeba.txt', 'txt', 'style', '5173', 'Heavy_Makeup')
insert 500 done
195 202599
('000196', 'Z:\\celebA\\data\\RGB\\rawdata\\img_celeba\\5564\\000196.jpg', '.jpg', 471, 407, 'RGB', 'Z:\\celebA\\data\\RGB\\labeldata\\img_celeba\\list_attr_celeba.txt', 'txt', 'Facial_Features', '5564', 'Big_Nose')
insert 500 done
265 202599
('000266', 'Z:\\celebA\\data\\RGB\\r